<a href="https://colab.research.google.com/github/SharmaineMangombe/Tunisian-Fraud-Detection-Challenge/blob/main/Tunisian_Fraud_Detection_Challenge_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import math

In [ ]:
Train=pd.read_csv('train.csv')
Test=pd.read_csv('test.csv')
SB=pd.read_csv('SampleSubmission (1).csv')
print(Train.shape)
print(Test.shape)
Train.head()

(25053, 121)
(10738, 120)


,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,...,TVA_CA132018,TVA_CA192018,ADB_MNTORD,SND_MNTPRD_I,SND_MNTTVA_I,SND_MNTDRC_I,SND_MNTAVA_I,SND_MNTTAX_I,SND_MNTPAY_I,SND_MNTAIR_I,SND_MNTPRD_E,SND_MNTTVA_E,SND_MNTDRC_E,SND_MNTAVA_E,SND_MNTTAX_E,SND_MNTPAY_E,SND_MNTAIR_E,SND_MNTPRD_A,SND_MNTTVA_A,SND_MNTDRC_A,SND_MNTAVA_A,SND_MNTTAX_A,SND_MNTPAY_A,SND_MNTAIR_A,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,id
0,49,14185,P,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,612500.0,0.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58912000.0,0.0,2530630.0,0.0,2013,13.575957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id0
1,221,6032,P,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3425295.0,0.0,10924985.0,0.0,2013,11.692894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id1
2,181,8907,P,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,...,0.0,0.0,34442746.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id2
3,252,5783,M,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,16.237871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id3
4,234,1477,P,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21604529.0,0.0,0.0,0.0,2015,13.399983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id4


In [ ]:
print(Train.info())
print(Test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25053 entries, 0 to 25052
Columns: 121 entries, BCT_CODBUR to id
dtypes: float64(103), int64(16), object(2)
memory usage: 23.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10738 entries, 0 to 10737
Columns: 120 entries, BCT_CODBUR to id
dtypes: float64(102), int64(16), object(2)
memory usage: 9.8+ MB
None


In [ ]:
pandas.set_option('display.max_rows', 121)
Test.isnull().sum()


In [ ]:
Train.dropna(axis=1, thresh=15000, inplace=True)
Y=Train['target']
Train.drop(['target','id'], axis=1, inplace=True)
TrainColumns= Train.columns.copy()
Test=Test[TrainColumns]

In [ ]:
print(Train.shape)
print(Test.shape)

(25053, 90)
(10738, 90)


In [ ]:
Train.isnull().sum()

BCT_CODBUR         0
CTR_MATFIS         0
CTR_CATEGO_X       0
FJU_CODFJU         0
CTR_CESSAT         0
ACT_CODACT         0
CTR_OBLDIR         0
CTR_OBLACP         0
CTR_OBLRES         0
CTR_OBLFOP         0
CTR_OBLTFP         0
CTR_OBLDCO         0
CTR_OBLTVA         0
CTR_OFODEC        10
CTR_OFODEP        10
CTR_OFODET        10
CTR_OBLAUT        10
CTR_OBLASS        10
CTR_ODTIMB        10
CTR_OBLTCL         0
CTR_OBLTHO        10
CTR_OBLDLI        10
CTR_OBLTVI        10
CTR_RATISS         0
EXE_EXERCI         0
TVA_CHAFF6      1974
TVA_CHAFF7      1974
TVA_CHAF10      1974
TVA_CHAF12      1974
TVA_CAF125      1974
TVA_CHAF15      1974
TVA_CHAF18      1974
TVA_CHAF22      1974
TVA_CHAF29      1974
TVA_CHAF36      1974
TVA_TOTDUE      1974
TVA_MOIDEB      1974
TVA_CRDINI      1974
TVA_BASIMB      1974
TVA_DEDIMB      1974
TVA_BASEQL      1974
TVA_DEDEQL      1974
TVA_BASEQI      1974
TVA_DEDEQI      1974
TVA_BASAUL      1974
TVA_DEDAUL      1974
TVA_BASAUI      1974
TVA_DEDAUI   

In [ ]:
Test.drop([TrainColumns],m)

In [ ]:
for i in Train.iloc[:,13:23].columns:
  Train[i].fillna(method='ffill', inplace=True)
  Test[i].fillna(method='ffill', inplace=True)

Train.fillna(0, inplace=True)
Test.fillna(0, inplace=True)
Train.isnull().sum()

BCT_CODBUR      0
CTR_MATFIS      0
CTR_CATEGO_X    0
FJU_CODFJU      0
CTR_CESSAT      0
               ..
AX2_HONORA      0
AX2_LOYERS      0
AX3_REVCAP      0
AX5_MNTMAR      0
RES_ANNIMP      0
Length: 89, dtype: int64

In [ ]:
print(Train['CTR_CATEGO_X'].unique())
Test['CTR_CATEGO_X'].unique()

['P' 'M' 'C' 'N']


array(['P', 'C', 'M'], dtype=object)

In [ ]:
Train=pd.get_dummies(Train,drop_first=True)
Test=pd.get_dummies(Test,drop_first=True)


In [ ]:
Train.drop('CTR_CATEGO_X_N',axis=1, inplace=True)
Train.head()

,BCT_CODBUR,CTR_MATFIS,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,TVA_BASEQL,...,TVA_TRSPOR,TVA_DEDREG,TVA_RESTIT,TVA_MNTPAY,TVA_MOIFIN,TVA_CRDFIN,TVA_ACHSUS,TVA_ACHEXO,TVA_ACHNDD,TVA_CAFEXP,TVA_CAFSUS,TVA_CAFEXO,TVA_AACHAB,TVA_CRDBLQ,TVA_CRDRES,TVA_CRDRMP,TVA_CRDREP,TVA_CRDRMC,TVA_MACHN,TVA_MCESQ,TVA_MTVLSM,TVA_TXVLSM,TVA_MTALSM,TVA_TXALSM,TVA_MTORDTVA,TVA_MTDJRTVA,TVA_MTORDTAX,TVA_MTDJRTAX,TVA_MTRESTAX,TVA_CA072018,TVA_CA132018,TVA_CA192018,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,CTR_CATEGO_X_M,CTR_CATEGO_X_P
0,49,14185,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,612500.0,0.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6412354.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58912000.0,0.0,2530630.0,0.0,2013,0,1
1,221,6032,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,720000.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3425295.0,0.0,10924985.0,0.0,2013,0,1
2,181,8907,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,31150.0,12.0,192751.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015,0,1
3,252,5783,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,1680387.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015,1,0
4,234,1477,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,1521555.0,...,0.0,0.0,0.0,224442.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21604529.0,0.0,0.0,0.0,2015,0,1


In [ ]:
from sklearn.preprocessing import MinMaxScaler
names=Train.columns.copy()
for i in names:
    scale = MinMaxScaler().fit(Train[[i]])
    
    Train[i] = scale.transform(Train[[i]])
    Test[i] = scale.transform(Test[[i]])


In [ ]:

from xgboost import XGBRegressor
model = XGBRegressor()
# fit the model
model.fit(Train,Y)
importance = model.feature_importances_
Zip_importance=zip(names, importance)
list(Zip_importance)

In [ ]:
less_important_XGBoost=['CTR_OBLDIR','CTR_OBLRES','CTR_OBLTFP','CTR_OBLDCO', 'CTR_OFODET','CTR_OBLAUT','CTR_OBLASS',
                'CTR_OBLTCL','CTR_OBLTHO','CTR_OBLTVI','TVA_CHAF10','TVA_CAF125','TVA_CHAF22','TVA_CHAF29',
                'TVA_CHAF36','TVA_BASIMB','TVA_DEDIMB','TVA_DEDEQL','TVA_BASEQI','TVA_TRSPOR','TVA_ACHSUS',
                'TVA_ACHEXO', 'TVA_ACHNDD','TVA_AACHAB','TVA_CRDRES','TVA_CRDRMP','TVA_CRDREP','TVA_TXVLSM',
                'TVA_MTALSM','TVA_TXALSM','TVA_MTORDTVA','TVA_MTORDTAX','TVA_MTDJRTAX','TVA_MTRESTAX',
                'TVA_CA072018','TVA_CA132018','TVA_CA192018','RES_ANNIMP', 'CTR_CATEGO_X_M','CTR_CATEGO_X_P']


In [ ]:
Train.drop(Train[less_important_XGBoost],axis=1, inplace=True) 
Test.drop(Test[less_important_XGBoost],axis=1, inplace=True) 
print(Train.shape)              
print(Test.shape)

(25053, 50)
(10738, 50)


In [ ]:
Train.head(10)

,BCT_CODBUR,CTR_MATFIS,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLACP,CTR_OBLFOP,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_ODTIMB,CTR_OBLDLI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF12,TVA_CHAF15,TVA_CHAF18,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASEQL,TVA_DEDEQI,TVA_BASAUL,TVA_DEDAUL,TVA_BASAUI,TVA_DEDAUI,TVA_BASRSM,TVA_DEDRSM,TVA_RSNRES,TVA_DEDREG,TVA_RESTIT,TVA_MNTPAY,TVA_MOIFIN,TVA_CRDFIN,TVA_CAFEXP,TVA_CAFSUS,TVA_CAFEXO,TVA_CRDBLQ,TVA_CRDRMC,TVA_MACHN,TVA_MCESQ,TVA_MTVLSM,TVA_MTDJRTVA,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR
0,0.057887,0.909114,1.000000,0.5,0.637614,1.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000259,0.0,5.335319e-07,0.000031,0.083333,0.000000,0.000000,0.0,1.850602e-07,1.424640e-07,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,5.989118e-05,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000650,0.0,0.000057,0.000000
1,0.306802,0.386553,1.000000,0.0,0.658872,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000124,0.0,0.000000,0.0,0.000000e+00,0.000003,0.083333,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,6.724777e-06,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000038,0.0,0.000245,0.000000
2,0.248915,0.570824,1.000000,0.0,0.637739,1.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.000000,0.0,0.000157,0.0,0.000000e+00,0.000019,0.083333,0.000000,0.000000,0.0,2.823856e-05,3.180533e-05,0.0,0.0,0.000045,0.000054,0.0,0.0,0.0,2.909400e-07,1.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,0.351664,0.370594,0.134831,0.0,0.425284,1.0,0.25,0.0,0.0,0.0,1.0,0.0,0.0,0.4,0.000000,0.0,0.000000,0.0,0.000000e+00,0.000000,0.083333,0.000001,0.000000,0.0,1.359883e-05,2.353710e-05,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,1.0,0.000040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,0.325615,0.094603,1.000000,0.0,0.663999,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.000000,0.0,0.000054,0.0,0.000000e+00,0.000006,0.083333,0.000000,0.000007,0.0,6.834005e-06,1.182838e-05,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,2.096281e-06,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000239,0.0,0.000000,0.000000
5,0.183792,0.536406,1.000000,0.0,0.426160,1.0,0.25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.001724,0.0,0.000000e+00,0.000203,0.083333,0.002422,0.000000,0.0,5.489274e-04,9.498575e-04,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,1.0,0.002580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
6,0.037627,0.070311,1.000000,0.0,0.476054,1.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
7,0.577424,0.838546,0.134831,0.0,0.469301,1.0,0.25,0.0,0.0,0.0,1.0,0.0,0.0,0.2,0.000000,0.0,0.000000,0.0,7.572405e-05,0.000076,0.083333,0.000000,0.000397,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000e+00,1.0,0.000024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000133
8,0.163531,0.893732,1.000000,0.0,0.400400,1.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.000000,0.0,0.000000,0.0,1.073129e-03,0.001072,0.083333,0.001511,0.000000,0.0,1.019394e-03,1.717913e-03,0.0,0.0,0.002784,0.001672,0.0,0.0,0.0,0.000000e+00,1.0,0.000153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
9,0.299566,0.671645,1.000000,0.0,0.663499,1.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.000000,0.0,0.000051,0.0,0.000000e+00,0.000006,0.083333,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.154420e-05,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(Train, Y, test_size=0.1)

In [ ]:

print('Random Forest', math.sqrt(mean_absolute_error(Y_test,RFR )))
print('Extra Tree', math.sqrt(mean_absolute_error(Y_test,ETR )))
print('LightGMB', math.sqrt(mean_absolute_error(Y_test,LGBM )))
print('CatBoost', math.sqrt(mean_absolute_error(Y_test,CBR)))
print('GradientBoost', math.sqrt(mean_absolute_error(Y_test,GBR )))

Random Forest 2.036693425094318
k Nebouring 2.3899393297540357
Extra Tree 2.0363149374739793
XGBoost 2.1514057490624774
Support Vector 2.1983065323056232


In [ ]:
# Best parameters

param_grid = {
        'n_estimators':[500,750],
        'max_depth': [ 8,9,10,12,15 ],
        'min_samples_split':[4,5,6],
        'max_features': ['auto', 'sqrt', 'log2']

        }
   
grid = RandomizedSearchCV(RFR_model,param_grid )      
grid.fit(X_train ,Y_train) 
print(grid.best_params_)

{'n_estimators': 750, 'min_samples_split': 4, 'max_features': 'auto', 'max_depth': 12}


In [ ]:
# Best parameters

param_grid = {
        'n_estimators':[250,400,500,750],
        'max_depth': [ 6,7,8 ],
        'min_samples_split':[5,6,7,8],
        'max_features': ['auto', 'sqrt', 'log2']

        }
   
grid = RandomizedSearchCV(RFR_model,param_grid )      
grid.fit(X_train ,Y_train) 
print(grid.best_params_)

In [ ]:
final_model2=ExtraTreesRegressor()
final_model2.fit(X_train, Y_train)
pred=final_model2.predict(X_test)
print('Exra tree', math.sqrt(mean_absolute_error(Y_test,pred)))

Random Forest 2.040396623601119


In [ ]:
prediction=final_model.predict(Test)
predcition=pd.DataFrame({'Client_id':SB['id'],'bank_account': prediction})
predcition.to_csv('prediction_RandomForest5.csv', index=False  )